# Simulator

In [3]:
import os
import pandas as pd


df = None
data = None

def load_data():
    global df
    excel_file_path = 'Output-Data/ETL-Prozess_Ergebnisse.xlsx'
    
    if os.path.exists(excel_file_path):
        df = pd.read_excel(excel_file_path, usecols=['Datum', 'SLP-Basisprozentwert', 'PV-10kWp', 'Strommarktpreis [€/kWh]'], nrows=35040)
        data = df[['Datum', 'SLP-Basisprozentwert']].copy()
        return data
    else:
        print(f"Error: Datei konnte nicht gefunden werden {excel_file_path}")
        return None, None

def simulate_power_consumption(data, annual_consumption, tariff_input):
    try:
        annual_consumption = float(annual_consumption)
        power_consumption = data['SLP-Basisprozentwert'] * annual_consumption / 100
        data['Stromverbrauch [kWh]'] = power_consumption
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation des Stromverbrauchs: {e}")
        return False
    
def simulate_power_production(data, pv_capacity):
    try:  
        if pv_capacity == 'PV-1kWp':
            power_production = df['PV-10kWp']/10
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-5kWp':
            power_production = df['PV-10kWp']/2
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-10kWp':
            power_production = df['PV-10kWp']
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-15kWp':
            power_production = (df['PV-10kWp']/2 + tmp['PV-10kWp'])
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-20kWp':
            power_production = df['PV-10kWp']*2
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-25kWp':
            power_production = (df['PV-10kWp']*2 + tmp['PV-10kWp']/2)
            data['Stromerzeugung [kWh]'] = power_production
        if pv_capacity == 'PV-30kWp':
            power_production = df['PV-10kWp']*3
            data['Stromerzeugung [kWh]'] = power_production
        
        return simulate_power_production_usage(data)
    except Exception as e:
        print(f"Fehler bei der Simulation der Stromerzeugung: {e}")
        return False 
    
def simulate_power_price(data, annual_consumption, current_tariff, current_base_price):
    global df 
    annual_consumption = float(annual_consumption)
    current_tariff = float(current_tariff)
    current_base_price = float(current_base_price)
        
    fixed_tariff_values = (data['Lasten Netz [kWh]'] * (current_tariff/100)) + ((current_base_price * 12) / 35040)
    variable_tariff_values = (data['Lasten Netz [kWh]'] * df['Strommarktpreis [€/kWh]']) + ((current_base_price * 12) / 35040)
        
    data['Fixer Stromtarif [€/kWh]'] = fixed_tariff_values
    data['Dynamischer Stromtarif [€/kWh]'] = variable_tariff_values
    data['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/kWh]']
    return True 
    
def simulate_grid_consumption(data, user_type):
    
    data['Lasten Netz [kWh]'] = 0.0
    
    if user_type == 'consumer':
        data['Lasten Netz [kWh]'] = data['Stromverbrauch [kWh]']
    else:
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            
            if user_type == 'prosumer':
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]']
                data.at[index,'Lasten Netz [kWh]'] =  lasten if lasten > 0 else 0.0
            if user_type == 'flexumer':
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]'] - data.at[index, 'Lasten Stromspeicher [kWh]']
                data.at[index,'Lasten Netz [kWh]'] = lasten if lasten > 0 else 0.0
    return True

def simulate_power_production_usage(data):
    try:
        data['Lasten Stromerzeugung [kWh]'] = 0.0
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            if production == 0:
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = 0.0
            else:
                lasten = consumption if (production-consumption) >= 0 else production
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = lasten
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation Lasten Stromerzeugung: {e}")
        return False
    
def simulate_battery_usage(data, battery_capacity):
    try:
        batt_capacity = 0.0
        if battery_capacity == '2,5 kWh':
             batt_capacity = 2.5
        if battery_capacity == '5,1 kWh':
             batt_capacity = 5.1
        if battery_capacity == '7,7 kWh':
             batt_capacity = 7.7
        if battery_capacity == '10,2 kWh':
             batt_capacity = 10.2
        if battery_capacity == '12,8 kWh':
             batt_capacity = 12.8
            
        battery_capacity = float(batt_capacity)

        data['Stromspeicher [kWh]'] = 0.0
        data['Lasten Stromspeicher [kWh]'] = 0.0
        
        batt_15mins_peak = batt_capacity / 4
     
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            
            if production == 0:
                batt_consumption = consumption if consumption <= batt_15mins_peak else batt_15mins_peak
                if batt_consumption <= battery_capacity:
                    data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = consumption
                    battery_capacity -= batt_consumption
                else:
                    data.at[index, 'Stromspeicher [kWh]'] = 0.0
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                    battery_capacity = 0.0
            else:
                if (production - consumption) > 0:
                    production_rest = batt_15mins_peak if (production - consumption) >= batt_15mins_peak else (production - consumption)
                    if (data.at[index - 1, 'Stromspeicher [kWh]'] + production_rest) <= batt_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity + production_rest
                        battery_capacity += production_rest
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = batt_capacity 
                        battery_capacity = batt_capacity
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = 0.0
                else:
                    batt_consumption = (consumption-production) if (consumption-production) <= batt_15mins_peak else batt_15mins_peak
                    if batt_consumption <= battery_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = batt_consumption
                        battery_capacity -= batt_consumption
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = 0.0
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                        battery_capacity = 0.0
                        
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation der Batterienutzung: {e}")
        return False
    
def run_simulator(annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity):
    simulate_power_consumption(data, annual_input, tariff_input)
    if user_type != 'consumer':
        simulate_power_production(data, pv_capacity)
    if user_type == 'flexumer':
        simulate_battery_usage(data, battery_capacity)
    simulate_grid_consumption(data, user_type)
    simulate_power_price(data, annual_input, tariff_input, base_input)